# 2023 Day 10: Pipe Maze

## Part 1

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

- | is a vertical pipe connecting north and south.
- \- is a horizontal pipe connecting east and west.
- L is a 90-degree bend connecting north and east.
- J is a 90-degree bend connecting north and west.
- 7 is a 90-degree bend connecting south and west.
- F is a 90-degree bend connecting south and east.
- . is ground; there is no pipe in this tile.
- S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

```
.....
.F-7.
.|.|.
.L-J.
.....
```

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```

If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

You can count the distance each tile in the loop is from the starting point like this:

```
.....
.012.
.1.3.
.234.
.....
```

In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here are the distances for each tile on that loop:

```
..45.
.236.
01.78
14567
23...
```

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [1]:
# Loading the pipe map (puzzle input) from file
with open('aoc-2023-day-10.txt', 'r') as f:
    pipe_map = f.read().splitlines()

# Setting up the sample data
sample_data = '''-L|F7
7S-7|
L|7||
-L-J|
L|-JF'''.splitlines()

# Setting the allowed directions
allowed_directions = {
    '|': ['up', 'down'],
    '-': ['left', 'right'],
    'L': ['up', 'right'],
    'J': ['left', 'up'],
    '7': ['left', 'down'],
    'F': ['down', 'right'],
    'S': ['up', 'down', 'left', 'right']
}

# Overwriting the pipe map with the sample data for testing
# pipe_map = sample_data

In [2]:
# Setting the allowed directions
allowed_directions = {
    '|': ['up', 'down'],
    '-': ['left', 'right'],
    'L': ['up', 'right'],
    'J': ['left', 'up'],
    '7': ['left', 'down'],
    'F': ['down', 'right'],
    'S': ['up', 'down', 'left', 'right']
}

In [3]:
# Extracting the starting point
x_coord = 0
found = False
for row in pipe_map:
    for i in row:
        if i == 'S':
            y_coord = row.index('S')
            found = True
            break
    if found:
        break
    x_coord += 1

starting_coords = (y_coord, x_coord)
print(f'Starting coordinate: {starting_coords}')

Starting coordinate: (88, 69)


In [4]:
def find_next_position(current_coords, max_row_len, max_col_len, already_visited):
    '''
    Finds the next position(s) based on the current coordinates
    
    Inputs:
        - current_coords (list): A small list containing the current coordinates
        - already_visited (set): A list of the coordinates that have already been visited
        - max_row_len (int): The maximum length of a row
        - max_col_len (int): The maximum length of the columns
    
    Returns:
        - next_coords (list): A list containing the next coordinates that can be travelled
    '''
    # Extracting information from the current coordinates
    current_y_coord = current_coords[0]
    current_x_coord = current_coords[1]

    # Extracting the current character at the current coordinates
    current_char = pipe_map[current_y_coord][current_x_coord]
    
    # Instantiating a list of coordinates to check
    coords_to_check = []
    
    # Adding the coords that can be visited from the current point
    if 'left' in allowed_directions[current_char]:
        if current_x_coord != 0:
            coords_to_check.append('left')
    
    if 'up' in allowed_directions[current_char]:
        if current_y_coord != 0:
            coords_to_check.append('up')
    
    if 'right' in allowed_directions[current_char]:
        if current_x_coord != max_row_len:
            coords_to_check.append('right')
    
    if 'down' in allowed_directions[current_char]:
        if current_y_coord != max_col_len:
            coords_to_check.append('down')
    
    # Instantiating a list of coords that can go next
    next_coords = []
    
    # Checking if we can go in each direction
    for direction in coords_to_check:
        
        if direction == 'left':
            next_x_coord = current_x_coord - 1
            next_y_coord = current_y_coord
            next_char = pipe_map[next_y_coord][next_x_coord]
            if 'right' in allowed_directions[next_char]:
                next_coords.append((next_y_coord, next_x_coord))
        
        elif direction == 'right':
            next_x_coord = current_x_coord + 1
            next_y_coord = current_y_coord
            next_char = pipe_map[next_y_coord][next_x_coord]
            if 'left' in allowed_directions[next_char]:
                next_coords.append((next_y_coord, next_x_coord))
        
        elif direction == 'up':
            next_x_coord = current_x_coord
            next_y_coord = current_y_coord - 1
            next_char = pipe_map[next_y_coord][next_x_coord]
            if 'down' in allowed_directions[next_char]:
                next_coords.append((next_y_coord, next_x_coord))
            
        elif direction == 'down':
            next_x_coord = current_x_coord
            next_y_coord = current_y_coord + 1
            next_char = pipe_map[next_y_coord][next_x_coord]
            if 'up' in allowed_directions[next_char]:
                next_coords.append((next_y_coord, next_x_coord))

    # Filtering out already visited coordinates
    next_coords = [i for i in next_coords if tuple(i) not in already_visited]
    
    return next_coords

# Getting the max row and columns lengths
max_col_len = len(pipe_map)
max_row_len = len(pipe_map[0])
find_next_position(starting_coords, max_row_len, max_col_len, [])

[(88, 68), (87, 69)]

In [5]:
# Getting the max row and columns lengths
max_col_len = len(pipe_map)
max_row_len = len(pipe_map[0])

# Instantiating a list of places to visit
places_to_visit = [starting_coords]

# Instantiating a set of places already visited
already_visited = {starting_coords}

# Checking all the places to visit from the starting point
while len(places_to_visit) != 0:
    
    # Getting the next place to check
    current_coords = places_to_visit.pop(0)
    already_visited.add(current_coords)
    
    # Getting the next positions to travel
    next_coords = find_next_position(current_coords, max_row_len, max_col_len, already_visited)
    
    # Appending the next coordinates in places to visit
    if len(next_coords) == 0:
        continue
    else:
        for coords in next_coords:
            places_to_visit.append(coords)
        
print(f'Number of steps taken: {len(already_visited) // 2}')

Number of steps taken: 6864


## Part 2
You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

```
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
```
The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```
In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

Here are just the tiles that are enclosed by the loop marked with I:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```
In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

349

In [6]:
# Loading the pipe map (puzzle input) from file
with open('aoc-2023-day-10.txt', 'r') as f:
    pipe_map = f.read().splitlines()

# Setting up the sample data
sample_data = '''FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L'''.splitlines()

# Setting the allowed directions
allowed_directions = {
    '|': ['up', 'down'],
    '-': ['left', 'right'],
    'L': ['up', 'right'],
    'J': ['left', 'up'],
    '7': ['left', 'down'],
    'F': ['down', 'right'],
    'S': ['up', 'down', 'left', 'right']
}

# Overwriting the pipe map with the sample data for testing
# pipe_map = sample_data

In [7]:
# Extracting the starting point
x_coord = 0
found = False
for row in pipe_map:
    for i in row:
        if i == 'S':
            y_coord = row.index('S')
            found = True
            break
    if found:
        break
    x_coord += 1

starting_coords = (y_coord, x_coord)
print(f'Starting coordinate: {starting_coords}')

Starting coordinate: (88, 69)


In [8]:
# Getting the max row and columns lengths
max_col_len = len(pipe_map)
max_row_len = len(pipe_map[0])

# Instantiating a list of places to visit
places_to_visit = [starting_coords]

# Instantiating a set of places already visited
already_visited = {starting_coords}

# Checking all the places to visit from the starting point
while len(places_to_visit) != 0:
    
    # Getting the next place to check
    current_coords = places_to_visit.pop(0)
    already_visited.add(current_coords)
    
    # Getting the next positions to travel
    next_coords = find_next_position(current_coords, max_row_len, max_col_len, already_visited)
    
    # Appending the next coordinates in places to visit
    if len(next_coords) == 0:
        continue
    else:
        for coords in next_coords:
            places_to_visit.append(coords)
        
print(f'Number of steps taken: {len(already_visited) // 2}')

Number of steps taken: 6864


In [9]:
# Count the number of "inversions" in a row
def count_invs(i, j):
    # Everything up to (but not including) j in line i
    line = pipe_map[i]
    count = 0
    for k in range(j):
        if not (i, k) in already_visited:
            continue
        count += line[k] in {"J", "L", "|"}

    return count

ans = 0
for i, line in enumerate(pipe_map):
    for j in range(max_row_len):
        if not (i, j) in already_visited:
            invs = count_invs(i, j)
            if invs % 2 == 1:
                ans += 1

print(ans)


349
